In [ ]:
def check_tuple(var, typ):
    if type(var)!=tuple and type(var)!=typ:
        return False 
    elif type(var)==tuple:
        if any(type(s)!=typ for s in var):
            return False
    return True

In [6]:
def diff_matrix_1d(grid_size, difference_order=4, periodic_boundary=False):
    """
    Return a differentiation matrices associated with derivative operators up to 2nd order on the line
    The derivative is calculated with respect to the equispaced grid
    The differentiation matrices are sparsed.
    
    Parameters 
    ----------
    grid_size : int nx
    difference_order : int default=4
        Controls order of the approximation of finite difference
    peryodic_boundary: boolean, default=False
        Sets whether periodic boundary conditions are imposed
    
    Returns
    ----------
    List of differentiation matrices of size nx X nx of rising order including identity. 
    I.e. (1, dx, d2x)
    
    References
    ---------------
    Uses sympy.calculus.finite_diff.finite_diff_weights
    """
    #----------------Check input------------------ 
    if any(type(inp)!=int for inp in (grid_size, difference_order)):
        raise ValueError("grid_size and difference_order must be integers")
    if type(periodic_boundary)!=bool:
        raise ValueError("periodic_boundary must be boolean")

    if difference_order+1 > grid_size:
        raise ValueError("Stencil is larger then the whole grid")
    
    #-------------Body------------------    
    
    h0=int(difference_order/2) #This is a number of neighbours
    stencil=[S(i) for i in range(-h0,h0+1,1)]
    
    d_weights = [np.array(finite_diff_weights(2, stencil, offset),dtype=float)[:,-1] for offset in range(-h0,h0+1,1)]
    
    return d_weights

In [7]:
dw=diff_matrix_1d(5)[3]

In [8]:
dw[2]

array([-0.08333333,  0.33333333,  0.5       , -1.66666667,  0.91666667])

In [9]:
np.version.version

'1.11.2'

In [30]:
t1=np.broadcast_to(dw[2],(3,5)).transpose()

In [31]:
t1.shape

(5, 3)

In [53]:
t1

array([[-0.08333333, -0.08333333, -0.08333333],
       [ 0.33333333,  0.33333333,  0.33333333],
       [ 0.5       ,  0.5       ,  0.5       ],
       [-1.66666667, -1.66666667, -1.66666667],
       [ 0.91666667,  0.91666667,  0.91666667]])

In [72]:
scipy.sparse.spdiags(t1,range(0,-5,-1),7,3).todense()

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


matrix([[-0.08333333,  0.        ,  0.        ],
        [ 0.33333333, -0.08333333,  0.        ],
        [ 0.5       ,  0.33333333, -0.08333333],
        [-1.66666667,  0.5       ,  0.33333333],
        [ 0.91666667, -1.66666667,  0.5       ],
        [ 0.        ,  0.91666667, -1.66666667],
        [ 0.        ,  0.        ,  0.91666667]])

In [195]:
def diff_matrix(grid_size, difference_order=4, periodic_boundary=False):
    """
    Return a differentiation matrices associated with derivative operators up to 2nd order.
    The derivative is calculated with respect to the equispaced multidimensional grid
    The differentiation matrices are sparsed, obtained by Kronecker product. 
    
    Parameters 
    ----------
    ! All tuples must be the same lenght equal to the number of grid dimensions.
    
    grid_size : tuple of ints (nx,ny,...)
    difference_order : tuple of ints (int,int,...), default=(5,5)
        Controls order of the approximation of finite difference for every axis.
    periodic_boundary: tuple of booleans, default=(False,False)
        Sets whether periodic boundary conditions are imposed in given direction
    
    Returns
    ----------
    List of differentiation matrices of size (nx*ny*...)x(nx*ny*...) of rising order including identity. 
    I.e. in 2 dimensions
??    (1, dx, dy, d2x, d2y, dxdy)
    
    References
    ---------------
    Uses sympy.calculus.finite_diff.finite_diff_weights
    and scipy.sparse.kron
    """
    #-----------Check input--------------
    #-----------------------------------
    def to_tuple(var):
    if type(var)!=tuple:
        return tuple([var]*len(grid_size))
    else:
        return var
            
    if not check_tuple(difference_order,int):
        raise ValueError("difference_order must be integer or tuple of integers")
    if not check_tuple(periodic_boundary,bool):
        raise ValueError("periodic_boundary must be integer or tuple of integers")
 
    (difference_order, periodic_boundary)=map(to_tuple, (difference_order, periodic_boundary))
    
    dims=tuple(len(inp) for inp in [grid_size,stencil_size,  periodic_boundary])
    if any(s1!=s2 for s1, s2 in zip(dims,dims[1:])):
        raise ValueError("Dimensions of grid, order and periodicity flags are not equal.")

    
    #-------Body-----------------------
    #-----------------------------------
    
    from sympy.calculus.finite_diff import finite_diff_weights
    from sympy import S
    
    h0=int(difference_order/2) #This is a number of neighbours
    stencil=[S(i) for i in range(-h0,h0+1,1)]
    res = finite_diff_weights(2, stencil, -3)
    res[0][-1]
    
    return res

In [197]:
diff_matrix((3,4))

TypeError: unsupported operand type(s) for /: 'tuple' and 'int'

In [64]:
stencil=[S(i) for i in range(-2,3,1)]
stencil

[-2, -1, 0, 1, 2]

In [65]:
h0=3
stencil=[S(i) for i in range(-h0,h0+1,1)]
res = finite_diff_weights(2, stencil, -3)
res[0][-1]

[1, 0, 0, 0, 0, 0, 0]

In [1]:
from sympy.calculus.finite_diff import finite_diff_weights
from sympy import S
import numpy as np

In [5]:
import scipy.sparse